# How-to: multybody nodes

Up until now we were creating nodes with a single body,
i.e. the internal logic of the node had only one implementation.

In this tutorial we shall learn how to add mutiple bodies to nodes and select them before execution.

## Why is this useful?

- User can define graph only once, and add and select
bodies later.

- Not all runtime and runtime simulators support HDL
languages such as Verilog. Instead an analogous
implementation can be used, e.g. Migen.

- HDL synthesis is a costly process takes a long time.
During development or debugging nodes' bodies with
HDL can be replaced by Python/C++.

## Simple example

Let's take a look at an example of a node that finds a sum of 2 integers.

In [1]:
import deltalanguage as dl


@dl.DeltaBlock(allow_const=False)
def add(a: int, b: int) -> dl.Void:
    print('Simple addition:')
    print(f'{a} + {b} = {a + b}')
    raise dl.DeltaRuntimeExit

with dl.DeltaGraph() as graph_1:
    add(1, 2)

This graph has 2 auto-generated nodes for constants and the main node:

In [2]:
print(graph_1)

DeltaGraph[main] {
    node[node_0]:
        ports:
            out:
                output, DInt32 -> add_2.a
        bodies:
            *PyConstBody
                tags: <lambda>

    node[node_1]:
        ports:
            out:
                output, DInt32 -> add_2.b
        bodies:
            *PyConstBody
                tags: <lambda>

    node[add_2]:
        ports:
            in:
                a, DInt32
                b, DInt32
        bodies:
            *PyFuncBody
                tags: add

}



Note that each node has only one body and its name is marked with `*`,
which means that it is selected.

By running it in our simulator we obtain the expected result:

In [3]:
dl.DeltaPySimulator(graph_1).run()

Simple addition:
1 + 2 = 3


Let's define and add a new body to the main node:

In [4]:
@dl.DeltaBlock(allow_const=False)
def new_add(a: int, b: int) -> dl.Void:
    print('Another addition:')
    print(f'{a} + {b} = {a + b}')
    raise dl.DeltaRuntimeExit

node = graph_1.find_node_by_name('add')
node.add_body(new_add)

We can see that a new body appears in the graph,
note that the original body is still selected as `*` has not moved:

In [5]:
print(graph_1)

DeltaGraph[main] {
    node[node_0]:
        ports:
            out:
                output, DInt32 -> add_2.a
        bodies:
            *PyConstBody
                tags: <lambda>

    node[node_1]:
        ports:
            out:
                output, DInt32 -> add_2.b
        bodies:
            *PyConstBody
                tags: <lambda>

    node[add_2]:
        ports:
            in:
                a, DInt32
                b, DInt32
        bodies:
            *PyFuncBody
                tags: add
            PyFuncBody
                tags: new_add

}



A body can be selected by using the following syntax:

In [6]:
node.select_body(preferred=['new_add'])

print(graph_1)

DeltaGraph[main] {
    node[node_0]:
        ports:
            out:
                output, DInt32 -> add_2.a
        bodies:
            *PyConstBody
                tags: <lambda>

    node[node_1]:
        ports:
            out:
                output, DInt32 -> add_2.b
        bodies:
            *PyConstBody
                tags: <lambda>

    node[add_2]:
        ports:
            in:
                a, DInt32
                b, DInt32
        bodies:
            PyFuncBody
                tags: add
            *PyFuncBody
                tags: new_add

}



Note, the new body is selected.
If the graph is executed again we shall see the new result:

In [7]:
dl.DeltaPySimulator(graph_1).run()

Another addition:
1 + 2 = 3


## Example with templates

Now we see that a node can have multiple bodies and
all of them must refer to the same I/O ports.

This might be an error-prone step, as we can accidentally
mis-name a port of simply forget to include one.

To minimise errors we propose to add bodies via templates, which ensure that
each body has exactly the same I/O ports.

We shall start by defining a template for a node that will do multiplication
2 integers:

In [8]:
from collections import OrderedDict

mult_template = dl.NodeTemplate(name="Multiplication",
                                in_params=OrderedDict([('a', int), ('b', int)]),
                                out_type=dl.Void)

Note that we define I/O ports of a template in the same way
as we would do using decorators `DeltaBlock`, `DeltaMethodBlock`,
or `Interactive`.
The only difference is the absence of a body.

In the next step multiple bodies are associated with this template:

In [9]:
@dl.DeltaBlock(mult_template, allow_const=False)
def mult(a: int, b: int) -> dl.Void:
    print('Simple multiplication:')
    print(f'{a} * {b} = {a * b}')
    raise dl.DeltaRuntimeExit

@dl.Interactive(template=mult_template,
                in_params=OrderedDict([('a', int), ('b', int)]),
                out_type=dl.Void)
def new_mult(node: dl.RealNode):
    a = node.receive('a')
    b = node.receive('b')
    print('Another multiplication:')
    print(f'{a} * {b} = {a * b}')
    raise dl.DeltaRuntimeExit

Let's create a graph where we use a node defined via the template
above:

In [10]:
with dl.DeltaGraph() as graph_2:
    node = mult(4, 3)

Note that the main node has the body that we used at construction
(marked with `*`), and also the other body associated with the template:

In [11]:
print(graph_2)

DeltaGraph[main] {
    node[node_3]:
        ports:
            out:
                output, DInt32 -> mult_5.a
        bodies:
            *PyConstBody
                tags: <lambda>

    node[node_4]:
        ports:
            out:
                output, DInt32 -> mult_5.b
        bodies:
            *PyConstBody
                tags: <lambda>

    node[mult_5]:
        ports:
            in:
                a, DInt32
                b, DInt32
        bodies:
            *PyFuncBody
                tags: mult
            PyInteractiveBody
                tags: new_mult

}



Each body can have multiple tags to help with selection.
The dafault tags are the name and the type of the node,
also more tags can be added at definition of the node.

Let's see which tags the selected body has:

In [12]:
node.body.access_tags

['mult',
 deltalanguage.wiring._node_classes.node_bodies.PyFuncBody,
 deltalanguage.wiring._node_classes.node_bodies.PythonBody,
 deltalanguage.wiring._node_classes.node_bodies.Body,
 abc.ABC,
 object]

Now let's select the other body and check that the tags are different:

In [13]:
node.select_body(preferred=['new_mult'])

node.body.access_tags

['new_mult',
 deltalanguage.wiring._node_classes.node_bodies.PyInteractiveBody,
 deltalanguage.wiring._node_classes.node_bodies.PyFuncBody,
 deltalanguage.wiring._node_classes.node_bodies.PythonBody,
 deltalanguage.wiring._node_classes.node_bodies.Body,
 abc.ABC,
 object]

On exectution the selected body is used indeed:

In [14]:
dl.DeltaPySimulator(graph_2).run()

Another multiplication:
4 * 3 = 12


## Bodyless node

Another reason why templates are important is the fact that the node's
body might be not available at the point when the graph is defined.
For instance the logic controlling an experimental set up
is under development and should be uploaded only right before runtime.

In order to do that the template itself can be used at the graph
construction:

In [15]:
sub_template = dl.NodeTemplate(name="Subtraction",
                               in_params=OrderedDict([('a', int), ('b', int)]),
                               out_type=dl.Void)


with dl.DeltaGraph() as graph_3:
    sub_template.call(10, 5)

The node will show that there aren't any bodies available, as expected:

In [16]:
print(graph_3)

DeltaGraph[main] {
    node[node_6]:
        ports:
            out:
                output, DInt32 -> template_Subtraction_8.a
        bodies:
            *PyConstBody
                tags: <lambda>

    node[node_7]:
        ports:
            out:
                output, DInt32 -> template_Subtraction_8.b
        bodies:
            *PyConstBody
                tags: <lambda>

    node[template_Subtraction_8]:
        ports:
            in:
                a, DInt32
                b, DInt32
        bodies:
            None

}

